In [1]:
# Reuse DSA code from previous

q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
p_hex = '800000000000000089e1855218a0e7dac38136ffafa72eda7' \
        '859f2171e25e65eac698c1702578b07dc2a1076da241c76c6' \
        '2d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebe' \
        'ac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2' \
        'b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc87' \
        '1a584471bb1'
g_hex = '5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119' \
        '458fef538b8fa4046c8db53039db620c094c9fa077ef389b5' \
        '322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a047' \
        '0f5b64c36b625a097f1651fe775323556fe00b3608c887892' \
        '878480e99041be601a62166ca6894bdd41a7054ec89f756ba' \
        '9fc95302291'

p = int(p_hex, base = 16)
g = int(g_hex, base = 16)

N = 40*4
L = len(p_hex)*4
assert (p-1) % q == 0

In [2]:
from hashlib import sha256

# Create a hash function with |H| = N bits output
def Hash(x: bytes, N = N):
    assert type(x) is bytes
    assert N <= 256
    digest_bytes = sha256(x).digest()
    digest_int = int.from_bytes(digest_bytes, byteorder = 'big')
    return digest_int & ( ( 0b1 << N ) - 1 )

assert Hash(b'abc') < 2**N

In [3]:
from random import randint

def DSAGeneratePrivateKey(p = p, q = q, g = g):
    return randint(1, q-1)

def DSAGeneratePublicKey(x, p = p, q = q, g = g ):
    assert type(x) is int and 1 <= x <= q-1
    return pow(g, x, p)

def DSASign(message, x, p = p, q = q, g = g, H = Hash):
    # x is the privkey
    assert type(message) is bytes
    assert type(x) is int and 1 <= x <= q-1
    
    k = randint(1, q-1)
    r = pow(g, k, p) % q
    
    k_inv = pow(k, q-2, q) # https://en.wikipedia.org/wiki/Fermat%27s_little_theorem
    assert (k*k_inv) % q == 1
    s = ( k_inv*( H(message) + x*r  ) ) % q
    
    assert s != 0 and r != 0
    signature = dict(r = r, s = s)
    signature.update(k = k) if leaky else None
    return signature
    

def DSAVerify(message, r, s, y, p = p, q = q, g = g, H = Hash):
    # y is the pubkey
    assert type(message) is bytes
    assert 0 < r < q 
    assert 0 < s < q
    
    w  = pow(s, q-2, q) # w is s inverse, again using FLT
    assert w*s % q == 1
    u1 = w*H(message) % q
    u2 = w*r % q
    
    # Using property (A * B) mod C = (A mod C * B mod C) mod C
    v  = pow(g, u1, p) * pow(y, u2, p)
    v %= p
    v %= q
    
    return True if v == r else False

def ModInv(x, p):
    # Using Fermat's little theorem
    y = pow(x, p-2, p)
    assert ( x*y ) % p == 1
    return y

# Attack function from the previous challenged, modified slightly
def AttackPrivateKey(s, k, m):
    r = pow(g, k, p) % q
    r_inv = pow(r, q-2, q) # FLT
    x = ( ( (s*k) - m ) * r_inv ) % q
    return x

In [4]:
# Read in data from file

import re

chunks = open('messages.txt').read().split('\n\n')
parsed = tuple( dict() for chunk in chunks )
for n, chunk in enumerate(chunks):
    lines = chunk.split('\n')
    assert len(lines) == 4
    for line in lines:
        if ( match := re.match('^msg: (.+)$', line ) ):
            parsed[n]['msg'] = match.group(1)
        elif ( match := re.match('^s: ([0-9]+)$', line) ):
            parsed[n]['s'] = int( match.group(1) )
        elif ( match := re.match('^r: ([0-9]+)$', line) ):
            parsed[n]['r'] = int( match.group(1) )
        elif ( match := re.match('^m: ([0-9a-f]+)$', line) ):
            parsed[n]['m'] = int(match.group(1), base = 16)
        else:
            raise Exception('Exactly one regex should have matched')
            
assert all( len(parsed_chunk) == 4 for parsed_chunk in parsed )

In [5]:
from itertools import combinations

# This is the pubkey corresponding to the privkey which signed the messages.  
# I'll use this to verify my answer is correct
target_pubkey = int('2d026f4bf30195ede3a088da85e398ef869611d0f68f07' \
                    '13d51c9c1a3a26c95105d915e2d8cdf26d056b86b8a7b8' \
                    '5519b1c23cc3ecdc6062650462e3063bd179c2a6581519' \
                    'f674a61f1d89a1fff27171ebc1b93d4dc57bceb7ae2430' \
                    'f98a6a4d83d8279ee65d71c1203d2c96d65ebbf7cce9d3' \
                    '2971c3de5084cce04a2e147821', base = 16
                   )

In [6]:
# The attack works for the following reason, 
# if two messages use the same k we have the following system of equations.
# s1*k mod q = m1 + x*r mod q
# s2*k mod q = m2 + x*r mod q
# subtract and factor...
# k*(s1 - s2) mod q = (m1 - m2) mod q

for message1, message2 in combinations(parsed, 2):
    m1, m2 = message1['m'], message2['m']
    s1, s2 = message1['s'], message2['s']
    
    m_diff = (m1-m2) % q
    s_diff = (s1-s2) % q
    
    k_guess = m_diff * ModInv(s_diff, q) % q
    privkey_guess = AttackPrivateKey(s1, k_guess, m1)
    pubkey_guess  = DSAGeneratePublicKey(privkey_guess)
    
    if pubkey_guess == target_pubkey:
        print('Private key found:', pubkey_guess)
        break
    

Private key found: 31606753313791117461614062426299082706319989724145856741134186114288533288243228304546692322192944781893730246986780372401161245868861852847563804533429202125604550548974600238762748043228702089788772143438263650534720332910581299414338429419098687610042294474438432307481975471529148310708599863811864033313
